# Setup

Pip install `ultralytics` and [dependencies](https://github.com/ultralytics/ultralytics/blob/main/pyproject.toml) and check software and hardware.

In [1]:
# %pip install ultralytics
import ultralytics
ultralytics.checks()

Ultralytics YOLOv8.2.60 🚀 Python-3.10.12 torch-2.3.1+cu121 CUDA:0 (NVIDIA A100-SXM4-40GB, 39327MiB)
Setup complete ✅ (22 CPUs, 117.9 GB RAM, 31.8/48.3 GB disk)


In [16]:
import cv2
from ultralytics import YOLO, solutions

# Load the pre-trained YOLOv8 model
model = YOLO("/root/med_si_track/opervu-240724-train/train/weights/best.pt")
print(model.names)
# Open the video file
cap = cv2.VideoCapture("/mnt/sdb/needle_images/output.mp4")
assert cap.isOpened(), "Error reading video file"

# Get video properties: width, height, and frames per second (fps)
w, h, fps = (int(cap.get(x)) for x in (cv2.CAP_PROP_FRAME_WIDTH, cv2.CAP_PROP_FRAME_HEIGHT, cv2.CAP_PROP_FPS))

# Define points for a line or region of interest in the video frame
line_points = [(20, 20),(20, 2048), (2048, 2048), (2048, 20)]  # Line coordinates

# Specify classes to count, for example: person (0) and car (2)
classes_to_count = [5]  # Class IDs for person and car

# Initialize the video writer to save the output video
video_writer = cv2.VideoWriter("object_counting_output.avi", cv2.VideoWriter_fourcc(*"mp4v"), fps, (w, h))

# Initialize the Object Counter with visualization options and other parameters
counter = solutions.ObjectCounter(
    view_img=True,  # Display the image during processing
    reg_pts=line_points,  # Region of interest points
    names=model.names,  # Class names from the YOLO model
    draw_tracks=True,  # Draw tracking lines for objects
    line_thickness=10,  # Thickness of the lines drawn
)

# Process video frames in a loop
while cap.isOpened():
    success, im0 = cap.read()
    if not success:
        print("Video frame is empty or video processing has been successfully completed.")
        break

    # Perform object tracking on the current frame, filtering by specified classes
    tracks = model.track(im0, persist=True, conf=0.1, iou=0.1, show=False, classes=classes_to_count, imgsz = 2480)

    # Use the Object Counter to count objects in the frame and get the annotated image
    im0 = counter.start_counting(im0, tracks)

    # Write the annotated frame to the output video
    video_writer.write(im0)

# Release the video capture and writer objects
cap.release()
video_writer.release()

# Close all OpenCV windows
cv2.destroyAllWindows()

{0: 'incision', 1: 'gauze', 2: 'glove', 3: 'black_suture', 4: 'vesiloop', 5: 'needle', 6: 'needle_holder', 7: 'sucker', 8: 'clamp', 9: 'forceps', 10: 'obstruction', 11: 'woodspack', 12: 'snare', 13: 'sponge', 14: 'scissors', 15: 'bovie', 16: 'scalpel'}
WARNING ⚠️ Environment does not support cv2.imshow() or PIL Image.show()
The DISPLAY environment variable isn't set.
Polygon Counter Initiated.

WARNING ⚠️ imgsz=[2480] must be multiple of max stride 32, updating to [2496]
0: 2496x2496 1 needle, 56.4ms
Speed: 26.4ms preprocess, 56.4ms inference, 1.2ms postprocess per image at shape (1, 3, 2496, 2496)

WARNING ⚠️ imgsz=[2480] must be multiple of max stride 32, updating to [2496]
0: 2496x2496 1 needle, 50.9ms
Speed: 26.3ms preprocess, 50.9ms inference, 1.2ms postprocess per image at shape (1, 3, 2496, 2496)

WARNING ⚠️ imgsz=[2480] must be multiple of max stride 32, updating to [2496]
0: 2496x2496 1 needle, 50.2ms
Speed: 26.4ms preprocess, 50.2ms inference, 1.2ms postprocess per image at s

In [15]:
import os
import cv2
from ultralytics import YOLO, solutions

# Load the pre-trained YOLOv8 model
model = YOLO("/root/med_si_track/opervu-240724-train/train/weights/best.pt")
print(model.names)

# Directory containing the images
image_directory = "/mnt/sdb/needle_images/"
output_directory = "./needle_images_output"

# Ensure the output directory exists
os.makedirs(output_directory, exist_ok=True)

# Define points for a line or region of interest in the image
line_points = [(20, 20),(20, 2048), (2048, 2048), (2048, 20)]  # Line coordinates

# Specify classes to count, for example: person (5)
classes_to_count = [5]  # Class IDs for person

# Initialize the Object Counter with visualization options and other parameters
counter = solutions.ObjectCounter(
    view_img=True,  # Display the image during processing
    reg_pts=line_points,  # Region of interest points
    names=model.names,  # Class names from the YOLO model
    draw_tracks=True,  # Draw tracking lines for objects
    line_thickness=10,  # Thickness of the lines drawn
    
)

# Process each image in the directory
for filename in os.listdir(image_directory):
    if filename.endswith(('.png', '.jpg', '.jpeg')):
        image_path = os.path.join(image_directory, filename)
        output_path = os.path.join(output_directory, filename)
        
        # Read the image
        im0 = cv2.imread(image_path)
        if im0 is None:
            print(f"Error reading image {image_path}")
            continue

        # Perform object tracking on the current image, filtering by specified classes
        tracks = model.track(im0, persist=True, conf=0.1, iou=0.1, show=False, classes=classes_to_count, imgsz=2496)

        # Use the Object Counter to count objects in the image and get the annotated image
        im0 = counter.start_counting(im0, tracks)

        # Save the annotated image to the output directory
        cv2.imwrite(output_path, im0)
        print(f"Processed and saved: {output_path}")

# Close all OpenCV windows
cv2.destroyAllWindows()


{0: 'incision', 1: 'gauze', 2: 'glove', 3: 'black_suture', 4: 'vesiloop', 5: 'needle', 6: 'needle_holder', 7: 'sucker', 8: 'clamp', 9: 'forceps', 10: 'obstruction', 11: 'woodspack', 12: 'snare', 13: 'sponge', 14: 'scissors', 15: 'bovie', 16: 'scalpel'}
WARNING ⚠️ Environment does not support cv2.imshow() or PIL Image.show()
The DISPLAY environment variable isn't set.
Polygon Counter Initiated.

0: 2496x2496 (no detections), 56.3ms
Speed: 25.9ms preprocess, 56.3ms inference, 0.8ms postprocess per image at shape (1, 3, 2496, 2496)
Processed and saved: ./needle_images_output/27505.jpg

0: 2496x2496 (no detections), 56.3ms
Speed: 25.8ms preprocess, 56.3ms inference, 0.8ms postprocess per image at shape (1, 3, 2496, 2496)
Processed and saved: ./needle_images_output/27508.jpg

0: 2496x2496 1 needle, 56.1ms
Speed: 28.2ms preprocess, 56.1ms inference, 1.2ms postprocess per image at shape (1, 3, 2496, 2496)
Processed and saved: ./needle_images_output/27504.jpg

0: 2496x2496 1 needle, 56.2ms
Spe